In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

## Part 1

In [96]:
df = pd.read_csv('C:\\Users\\junge\\Desktop\\Python\\Lab_2\\Telecust1.csv')

# get rid of char table
df = df.drop('custcat', axis = 1)
df = df.fillna(df.mean())
feature_names = ['tenure', 'age', 'income', 'marital', 'address', 'ed', 'employ', 'retire', 'gender', 'reside']
df.head()

,region,tenure,age,income,marital,address,ed,employ,retire,gender,reside
0,2,13,44,64,1,9,4,5,0,0,2
1,3,11,33,136,1,7,5,5,0,0,6
2,3,68,52,116,1,24,1,29,0,1,2
3,2,33,33,33,0,12,2,0,0,1,1
4,2,23,30,30,1,9,1,2,0,0,4


In [98]:
X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df['region'], test_size=0.2, random_state=0)
# scale data to improve accuracy
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [99]:
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ]
        }
       ]

In [100]:
%%capture --no-display
clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,
                           scoring='accuracy')
# fit model
clf.fit(X_trainscaled,y_train)

GridSearchCV(cv=3, estimator=MLPClassifier(),
             param_grid=[{'activation': ['identity', 'logistic', 'tanh',
                                         'relu'],
                          'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,),
                                                 (6,), (7,), (8,), (9,), (10,),
                                                 (11,), (12,), (13,), (14,),
                                                 (15,), (16,), (17,), (18,),
                                                 (19,), (20,), (21,)],
                          'solver': ['lbfgs', 'sgd', 'adam']}],
             scoring='accuracy')

In [101]:
# best params and accuracy
print('Best params:',clf.best_params_)
print("Test set: ", clf.score(X_testscaled,y_test))

Best params: {'activation': 'identity', 'hidden_layer_sizes': (2,), 'solver': 'sgd'}
Test set:  0.305


## Part 2

In [103]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [104]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 3s 1us/step


In [105]:
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(64, 3, activation="relu")(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(10)(x)
cnn_model = keras.Model(inputs=inputs, outputs=outputs, name="cnn_model")
cnn_model.summary()

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
flatten (Flatten)            (None, 43264)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                432650    
Total params: 433,546
Trainable params: 433,418
Non-trainable params: 128
_________________________________________________________________


In [106]:
cnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)
# training
cnn_model.fit(x_train, y_train, batch_size=256, epochs=5, validation_split=0.2)

Epoch 1/5
188/188 [==============================] - 72s 378ms/step - loss: 2.3051 - accuracy: 0.7632 - val_loss: 0.8201 - val_accuracy: 0.8107
Epoch 2/5
188/188 [==============================] - 68s 360ms/step - loss: 0.4350 - accuracy: 0.8920 - val_loss: 0.5665 - val_accuracy: 0.8671
Epoch 3/5
188/188 [==============================] - 67s 357ms/step - loss: 0.2650 - accuracy: 0.9231 - val_loss: 0.6773 - val_accuracy: 0.8579
Epoch 4/5
188/188 [==============================] - 67s 355ms/step - loss: 0.1868 - accuracy: 0.9427 - val_loss: 0.5816 - val_accuracy: 0.8832
Epoch 5/5
188/188 [==============================] - 67s 354ms/step - loss: 0.1440 - accuracy: 0.9538 - val_loss: 0.6763 - val_accuracy: 0.8810


In [107]:
test_scores = cnn_model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

313/313 - 2s - loss: 0.7162 - accuracy: 0.8728
Test loss: 0.716224193572998
Test accuracy: 0.8727999925613403


## Part 3

In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [6]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to C:\Users\junge\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [7]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))
vocab = np.array(encoder.get_vocabulary())

In [9]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [10]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [11]:
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/3
391/391 [==============================] - 780s 2s/step - loss: 0.6858 - accuracy: 0.5102 - val_loss: 0.5029 - val_accuracy: 0.7339
Epoch 2/3
391/391 [==============================] - 772s 2s/step - loss: 0.4761 - accuracy: 0.7447 - val_loss: 0.4774 - val_accuracy: 0.7411
Epoch 3/3
391/391 [==============================] - 706s 2s/step - loss: 0.4033 - accuracy: 0.8146 - val_loss: 0.3757 - val_accuracy: 0.8328


In [12]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 62s 159ms/step - loss: 0.3646 - accuracy: 0.8426
Test Loss: 0.36456143856048584
Test Accuracy: 0.8425999879837036


In [13]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/3
391/391 [==============================] - 970s 2s/step - loss: 0.6843 - accuracy: 0.5100 - val_loss: 0.4231 - val_accuracy: 0.8000
Epoch 2/3
391/391 [==============================] - ETA: 0s - loss: 0.3933 - accuracy: 0.8282

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))